# Preprosess nowac dataset

Notebook showing how the nowac dataset preprocessed.

This is not based on the original dataset, but instead on Gudmund's 'nowac-1.2_old.txt'.

## Suggestions from Gudmund
1) Første ord skal ikke være et tall ‘sekvens', e.g. 12.34.56, men kan muligens være kun ett tall?

2) Første ord skal ha lengde > 1 med mindre det er I eller Å, og det må ha stor forbokstav.

3) Setningen skal ha minst ett (kanskje 2) ord fra de første 1000 ord fra 'ord10k_utf-8.txt’ (tror jeg fjernet tall fra den liste, mulig det er noe annet rart der også).

4) Andel ord med en karakter skal ikke være med enn 20 prosent av setningen eller flere enn 15 stykker totalt.

5) Det skal ikke være for mange ord av store bokstaver, for å fjerne slike ’… send STOPP til ...’

6) Det skal ikke være for mange 'tall ord' i setningen (mindre enn 3), men da teller jeg ikke årstall.

7) Hvis det er mer enn 5 ord i en setning med stor bokstav kaster jeg dem ut, mulig dette skal være en prosent av ord og ikke antall.

### Additional

8) Remove all sentences with words that are a mix between letters and non-letters (numbers, signs, etc). This could remove comments "hei", but for now, there are spaces inbetween.

## Comments

- There is always a space before punctuation. E.g. "hei , sa han ."
- It looks like lines ends with " \n"

In [1]:
# run unittests
! ipython test_matchText.py

..........
----------------------------------------------------------------------
Ran 10 tests in 0.000s

OK


In [2]:
from exceptions import NotImplementedError
import re

**Encodings:**
Under we set up encodings so we can use regex on the norwegian language.

In [3]:
import codecs #used to set utf-8 enconding, so we get unicode strings
import locale
locale.setlocale(locale.LC_ALL, 'no_NO') #so \w, \W, etc. contains norwegian letters

'no_NO'

In [4]:
from matchText import MatchText
# execfile('matchText.py')

## Functions

First we read a chunk of the data:

In [5]:
def readChunk(handle, size=100):
    """Read a chunk of a file"""
    chunk = []
    for i, line in enumerate(handle):
        chunk.append(line)
        if i+1 == size:
            break
    return chunk

Then we give criteria for which lines should be included:

In [6]:
def processChunk(chunk):
    removeIdx = set()
    startNr = re.compile(r'\A\d+\s+')
    for i, line in enumerate(chunk):
        line = MatchText(startNr.sub('', line))
        if not line.startWithAWord():
            removeIdx.add(i)
            continue
        # This is covered by startWithAWord
#         if line.startWithSequence():
#             removeIdx.add(i)
#             continue
        #Here we have a problem with punctuation separated by whitespace.
        if line.nrOf1CharacterWords() >= 15:
            removeIdx.add(i)
            continue
        if line.nrOf1CharacterWords() / float(line.nrOfWords()) > 0.2:
            removeIdx.add(i)
            continue
        if line.nrOfUpperCaseWords() >= 3:
            removeIdx.add(i)
            continue
        if (line.nrOfNumbers() - line.nrOfYears()) >= 3:
            removeIdx.add(i)
            continue
        if line.nrOfWordsWithUpperCase() >= 5:
            removeIdx.add(i)
            continue
#         if line.nrOfMixedWords() > 0:
#             removeIdx.add(i)
#             continue
        if line.hasNrOf1000CommonNorwegianWords(1) == False:
            removeIdx.add(i)
            continue
    text = u''
    removedText = u''
    for i, line in enumerate(chunk):
        if i in removeIdx:
            removedText += line
        else:
            text += line
    return text, removedText

**The order the evaluation in 'processChunk' is done, shold be considered!!!!**

Then we write the lines that passed and didn't pass the tests to two separeate files:

In [7]:
def writeChunk(text, removedText, handle, handleRemove):
    handle.write(text)
    handleRemove.write(removedText)

## Execution

The size of the file is 3.9 GB

In [8]:
! du -h nowac-1.2_old.txt

3.9G	nowac-1.2_old.txt


With 33.5 million sentences:

In [9]:
%%time
! wc nowac-1.2_old.txt

  33501699  721718825 4085354972 nowac-1.2_old.txt
CPU times: user 1.24 s, sys: 206 ms, total: 1.44 s
Wall time: 5min 53s


And here is the acutual execution! By some reason, reading chunks of size 100 is the most efficient.

In [10]:
filename = 'nowac-1.2_old.txt'

In [11]:
# Uncomment for simple test case
# filename = 'test.txt'
# ! head -n 100000 nowac-1.2_old.txt > test.txt

In [12]:
%%time
chunk = ['']
with codecs.open(filename, encoding='utf-8') as inF, \
     codecs.open('nowac_cleaned.txt', 'w', encoding='utf-8') as outF, \
     codecs.open('nowac_removed.txt', 'w', encoding='utf-8') as outFRemov:
    while len(chunk) > 0:
        chunk = readChunk(inF, 100)
        text, removedText = processChunk(chunk)
        writeChunk(text, removedText, outF, outFRemov)

CPU times: user 32min 30s, sys: 6.47 s, total: 32min 36s
Wall time: 38min 25s


The data is split in a cleaned file and a rejected file:

In [13]:
! wc nowac_cleaned.txt
! wc nowac_removed.txt

  22419428  413486154 2371748257 nowac_cleaned.txt
  11082271  308232671 1713606715 nowac_removed.txt


In [14]:
! head -n 10 nowac_cleaned.txt

2 Alexei Nikolaevich , Tsarevich of Russia . 
5 Inspirert av temaet i " Iskaldt " , gikk produsenten i gang med å skaffe seg miljøalibi . 
6 En gang var Norge og Danmark faktisk Russland allierte . 
7 Komiteen mente riktignok at det var urealistisk å bygge opp kostnadskrevende teknologisk og naturvitenskapelig forskning . 
8 Men Haatuft poengterer at « ski inn-ski ut » ikke er avgjørende for prisen på en hytte på Geilo . 
9 Eneste minuset er vel at C K bytter kusk , men det tror jeg får liten innvirkning . 
12 Jeg ble ikke sittende ved siden av dem i lunsjen , jeg dultet ikke borti dem under minglingen på mottagelsene . 
13 En forutsigbarhetstilnærming som den tilpasningsorienterte skole vil hevde er håpløs på grunn av foranderlige krav . 
14 Boken går inn i problematikken rundt lese - og skrivevansker , og tar opp en rekke spørsmål og drøfter disse i lys av nyere forskning . 
15 De kommissariske statsrådene , som sorterte direkte under rikskommissar Terboven , og NS på sin side , gikk

In [15]:
! head -n 10 nowac_removed.txt

1 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 10.11 RYG-STN , 11.11 STN-RYG , 22.12 OSL-AES , 27.12 AES-OSL . 
3 I potten er det kanoner som Liverpool , Manchester United , Inter og Ajax , for å nevne noen . 
4 På den deltar også arkitekt Niels Torp og konserndirektøren Kurt Ritter i hotellkjeden Rezidor . 
10 I ' m Cliff Richard , you know . 
11 men ' slapp ' Nja , ikke enig . 
29 I dag kommer CPI , som det vil bli mye fokus på etter en core PPI godt over forvetningene forrige fredag . 
30 Les mer ( 06.01.2008 ) ADRA bistår fordrevne i Kenya Urolighetene etter forrige ukes presidentvalg i Kenya tvinger hundretusener fra hjemmene sine . 
32 Forvant også andre forsterkninger fra 1 laget , som Korkmaz , Tsomou , Rivero , Heller og Toski og da stiller man meget sterkt i denne , også offensivt med trioen Tsmou , Heller og Rivero . 
35 Mvh Frank Skrevet : 3.4.2008 kl. 10:05 Svar Sitér David O. Takk ... Ja det var alt for lys ... Vi var på veien til Galleri på sondag men måte snur på grunn av tid pr